In [1]:
import torch
import nnsight
from PIL import Image
import os, json, argparse, itertools, math, torch, numpy as np
from tqdm import tqdm
from transformers import AutoProcessor
from vllm.inputs import TextPrompt
from nnsight.modeling.vllm import VLLM
import einops
import json

torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [2]:
model_name = "llava-hf/llava-1.5-7b-hf"
model = VLLM(model_name, device='cuda', dispatch=True)
processor = AutoProcessor.from_pretrained(model_name)

print(model.language_model.model.layers[0].self_attn.qkv_proj)

INFO 05-05 20:04:36 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='llava-hf/llava-1.5-7b-hf', speculative_config=None, tokenizer='llava-hf/llava-1.5-7b-hf', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=llava-hf/llava-1.5-7b-hf, num_scheduler_steps=1, chunked_prefill_enabled=False multi_step_stream_outputs=True, enable_prefix_caching=False, use_async_outpu

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


INFO 05-05 20:04:39 model_runner.py:1077] Loading model weights took 13.1342 GB
INFO 05-05 20:04:40 worker.py:232] Memory profiling results: total_gpu_memory=139.72GiB initial_memory_usage=13.77GiB peak_torch_memory=13.64GiB memory_usage_post_profile=13.88GiB non_torch_memory=0.71GiB kv_cache_size=111.39GiB gpu_memory_utilization=0.90
INFO 05-05 20:04:40 gpu_executor.py:113] # GPU blocks: 14257, # CPU blocks: 512
INFO 05-05 20:04:40 gpu_executor.py:117] Maximum concurrency for 4096 tokens per request: 55.69x
INFO 05-05 20:04:41 model_runner.py:1400] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 05-05 20:04:41 model_runner.py:1404] If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization` or switching to eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.


In [3]:

print(model.language_model.model.layers[0].self_attn.o_proj)

RowParallelLinear(input_features=4096, output_features=4096, bias=False, tp_size=1, reduce_results=True)


In [6]:
with model.trace() as tr, tr.invoke(TextPrompt(prompt="Hello", multi_modal_data={'image': Image.open("test.png")})):
    qkv = model.language_model.model.layers[0].self_attn.qkv_proj.output.save()


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  4.12it/s, est. speed input: 8.25 toks/s, output: 66.01 toks/s]


In [7]:
print(qkv)

(tensor([[ 8.8196e-02, -2.7979e-01,  5.8545e-01,  ..., -5.6763e-03,
         -2.1347e-02,  3.1018e-04],
        [ 1.6289e+00, -2.1094e+00,  9.0576e-01,  ...,  4.6349e-03,
         -8.3237e-03,  8.4000e-03]], device='cuda:0', dtype=torch.float16), None)


In [9]:
img = Image.open("/root/multimodal-interpretability/scripts/dataset/clean-1.jpg")
with model.trace() as tr, tr.invoke(TextPrompt(prompt="Hello", multi_modal_data={'image': img})):
    qkv = model.language_model.model.layers[0].self_attn.qkv_proj.output.save()


INFO 05-05 20:08:15 preprocess.py:215] Your model uses the legacy input pipeline instead of the new multi-modal processor. Please note that the legacy pipeline will be removed in a future release. For more details, see: https://github.com/vllm-project/vllm/issues/10114
WARNING 05-05 20:08:15 utils.py:445] The number of multi-modal placeholder tokens in the prompt is less than the number of multi-modal inputs. Extra placeholder tokens will be treated as plain text


NNsightError: Multi-modal placeholders and items must have the same length.